In [1]:
#Handle necessary imports
import pandas as pd
import numpy as np

from Bio import SeqIO, SeqUtils

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
#Load fasta sequence data
cyto = 'data/cyto.fasta.txt'
mito = 'data/mito.fasta.txt'
nucleus = 'data/nucleus.fasta.txt'
secreted = 'data/secreted.fasta.txt'

In [3]:
#Load 'cyto' data into dataframe and add label
with open(cyto) as file:  
    identifiers = []
    sequences = []
    for seq_record in SeqIO.parse(file, 'fasta'):  
        identifiers.append(seq_record.id)
        sequences.append(seq_record.seq)

cyto_df = pd.DataFrame.from_records(sequences)
cyto_df['label'] = 'cyto'

#Replace None entries with NaN: useful for features later
cyto_df.fillna(value=np.nan, inplace=True)

In [4]:
#Load 'mito' data into dataframe and add label
with open(mito) as file:  
    identifiers = []
    sequences = []
    for seq_record in SeqIO.parse(file, 'fasta'):  
        identifiers.append(seq_record.id)
        sequences.append(seq_record.seq)

mito_df = pd.DataFrame.from_records(sequences)
mito_df['label'] = 'mito'

#Replace None entries with NaN: useful for features later
mito_df.fillna(value=np.nan, inplace=True)

In [5]:
#Load 'nucleus' data into dataframe and add label
with open(nucleus) as file:  
    identifiers = []
    sequences = []
    for seq_record in SeqIO.parse(file, 'fasta'):  
        identifiers.append(seq_record.id)
        sequences.append(seq_record.seq)

nucleus_df = pd.DataFrame.from_records(sequences)
nucleus_df['label'] = 'nucleus'

#Replace None entries with NaN: useful for features later
nucleus_df.fillna(value=np.nan, inplace=True)

In [6]:
#Load 'secreted' data into dataframe and add label
with open(secreted) as file:  
    identifiers = []
    sequences = []
    for seq_record in SeqIO.parse(file, 'fasta'):  
        identifiers.append(seq_record.id)
        sequences.append(seq_record.seq)

secreted_df = pd.DataFrame.from_records(sequences)
secreted_df['label'] = 'secreted'

#Replace None entries with NaN: useful for features later
secreted_df.fillna(value=np.nan, inplace=True)

In [70]:
#Concatanate the dataframes into one with all the sequence data
full_data=pd.concat([cyto_df,mito_df,nucleus_df,secreted_df],0,ignore_index=True)

In [71]:
#Reindex the label column as the last column
cols = full_data.columns.tolist()
cols.insert(np.shape(full_data)[1], cols.pop(cols.index('label')))
full_data = full_data.reindex(columns= cols)
full_data.head()

,0,1,2,3,4,5,6,7,8,9,...,13091,13092,13093,13094,13095,13096,13097,13098,13099,label
0,M,G,Q,Q,V,G,R,V,G,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cyto
1,M,A,L,E,P,I,D,Y,T,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cyto
2,M,N,Q,I,E,P,G,V,Q,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cyto
3,M,S,E,E,P,T,P,V,S,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cyto
4,M,G,D,W,M,T,V,T,D,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cyto


In [72]:
#Remove labels column
labels = full_data.pop('label')
np.shape(full_data)

(9222, 13100)

In [166]:
#Feature engineering starts here 
#i.e. find relevant features for the sequence data

# 1. Sequence length
sequence_length = pd.DataFrame(full_data.count(axis=1),columns=['seq len'])
sequence_length = sequence_length.astype(float)

In [248]:
# 2. Global amino acid count i.e. no of amino acid per sequence
global_count = full_data.T

d = {}
for i in range(len(full_data)):
    series_global = global_count.groupby(i).size()
    d[str(i)]=pd.DataFrame({i:series_global.values},index = series_global.index+' global')

global_counts_feats = pd.concat([d[str(i)] for i in range(len(d))],axis=1)
global_counts_feats = global_counts_feats.fillna(0)
global_counts_feats = global_counts_feats.T

In [253]:
# 3. First 50 amino acid count 
local_count_first50 = global_count.drop(global_count.index[50:])

d = {}
for i in range(len(full_data)):
    series_global = local_count_first50.groupby(i).size()
    d[str(i)]=pd.DataFrame({i:series_global.values},index = series_global.index+' first50')

first50_counts_feats = pd.concat([d[str(i)] for i in range(len(d))],axis=1)
first50_counts_feats = first50_counts_feats.fillna(0)
first50_counts_feats = first50_counts_feats.T

In [250]:
# 4. Last 50 amino acid count 
local_count_last50 = global_count.drop(global_count.index[:50])

d = {}
for i in range(len(full_data)):
    series_global = local_count_last50.groupby(i).size()
    d[str(i)]=pd.DataFrame({i:series_global.values},index = series_global.index+' last50')

last50_counts_feats = pd.concat([d[str(i)] for i in range(len(d))],axis=1)
last50_counts_feats = last50_counts_feats.fillna(0)
last50_counts_feats = last50_counts_feats.T

In [ ]:
# 5. Isoelectric points


In [254]:
# Final feature dataframe to be used for model
features_df = pd.concat([sequence_length,global_counts_feats,first50_counts_feats,last50_counts_feats],1)
features_df.head()

,seq len,A global,B global,C global,D global,E global,F global,G global,H global,I global,...,P last50,Q last50,R last50,S last50,T last50,U last50,V last50,W last50,X last50,Y last50
0,1182.0,97.0,0.0,17.0,51.0,81.0,35.0,96.0,26.0,30.0,...,91.0,39.0,55.0,115.0,71.0,0.0,74.0,13.0,0.0,32.0
1,592.0,56.0,0.0,1.0,47.0,75.0,17.0,22.0,4.0,28.0,...,62.0,20.0,15.0,46.0,17.0,0.0,21.0,7.0,0.0,8.0
2,894.0,75.0,0.0,10.0,54.0,92.0,28.0,35.0,23.0,66.0,...,26.0,50.0,58.0,37.0,41.0,0.0,31.0,13.0,0.0,22.0
3,861.0,68.0,0.0,8.0,58.0,81.0,33.0,34.0,17.0,52.0,...,15.0,49.0,26.0,60.0,42.0,0.0,47.0,9.0,0.0,19.0
4,614.0,20.0,0.0,14.0,33.0,64.0,15.0,21.0,13.0,25.0,...,11.0,43.0,34.0,52.0,27.0,0.0,24.0,7.0,0.0,13.0


In [255]:
# Encoding the 4 class labels
le = preprocessing.LabelEncoder()
le.fit(labels.values)

labels_enc=le.transform(labels)
labels_df = pd.DataFrame(labels_enc,columns=['labels'])

In [256]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(features_df, labels_enc, random_state=0, test_size=0.3)

In [263]:
# Train a model
model = LogisticRegression()
y_pred = model.fit(X_train, y_train).predict(X_test)
print('Accuracy =', metrics.accuracy_score(y_test, y_pred))

Accuracy = 0.628478496567


In [258]:
# Confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

[[391  50 386  68]
 [ 59 270  49  17]
 [222  31 704  39]
 [ 26  40  41 374]]


In [259]:
# Evaluation Metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.56      0.44      0.49       895
          1       0.69      0.68      0.69       395
          2       0.60      0.71      0.65       996
          3       0.75      0.78      0.76       481

avg / total       0.63      0.63      0.62      2767

